### TODO
- Assign projects unique colors and use them everywhere
- DONE Setup Keyboard shortcut to view plots
- Set up minutely regeneration of plots
- Show date in bar plots
- Implement time_entry_days = diskcache.Cache(f"{project_dir}/time_entry_days.cache")
- Implement a goal accumulative graph
  - Implement a graph that goes further back (and generalize other code so it goes further back in time)

In [1]:
CLEAR_NAME_CACHE = False
CLEAR_PLOTS = False

In [2]:
import requests
import base64
import json
import gc
import os
import time
import datetime as dt
from datetime import datetime, date
import pytz
import diskcache
import urllib.parse
from matplotlib import pyplot as plt
plt.rcParams.update({'font.family': 'Noto Sans CJK JP'})
import numpy as np
from operator import itemgetter, attrgetter
import pandas as pd
from pdb import set_trace
from IPython.display import clear_output
from glob import glob as glob
import math

In [3]:
if CLEAR_PLOTS:
    for f in glob("plots/*"):
        os.remove(f)

# Functionality

## Utils

In [4]:
def now():
    return datetime.utcnow().replace(tzinfo=pytz.utc)

def filter_dict(d, keys):
    return list(map(lambda x: {k: v for k, v in x.items() if k in keys}, d))

## Setting up Toggl API
Note that a safe API request rate is 1 request per second.

In [5]:
project_dir = os.path.abspath('')

API_version = 'v9'
prefix = f"https://api.track.toggl.com/api/"

API = {
    "time_entries" :          prefix + "v8" + "/time_entries",
    "get_current_time_entry": prefix + API_version + "/me/time_entries/current", 
    "me_info" :               prefix + API_version + "/me",
    "projects_me" :           prefix + API_version + "/me/projects",
    "projects_ws" :           prefix + API_version + "/workspaces/4143224/projects",
    "workspaces" :            prefix + API_version + "/workspaces",
}

with open(f"{project_dir}/api_key.txt") as f:
    API_token = f"{f.read()}:api_token"
API_token_base64 = base64.b64encode(API_token.encode('ascii')).decode('ascii')

project_id_names = diskcache.Cache(f"{project_dir}/project_id_names.cache")
if CLEAR_NAME_CACHE:
    project_id_names.clear()

In [6]:
def fetch(url, headers={}):
    headers={"Content-Type": "application/json", 
                                    "Host": "api.track.toggl.com",
                                    "authorization": f"Basic {API_token_base64}", 
                                    **headers}
    r = requests.get(url, headers=headers)
    return json.loads(r.content.decode('utf-8'))

def fetch_project(project_id):
    return fetch(f"{API['projects_ws']}/{project_id}")

In [7]:
def update_current_time_entry_duration(time_entry):
    time_entry["duration"] = (datetime.utcnow().replace(tzinfo=pytz.utc) - \
                              time_entry["start"]).seconds
    return time_entry

def get_time_entries(start_date, end_date):
    date_to_str = lambda date: date.strftime('%Y-%m-%dT%H:%M:%S') + f'{date.strftime("%z")[:3]}:{date.strftime("%z")[3:]}'
    url_exts = '?'
    url_exts += f"start_date={urllib.parse.quote(date_to_str(start_date))}"
    url_exts += f"&end_date={urllib.parse.quote(date_to_str(end_date))}"
    time_entries = fetch(API['time_entries'] + url_exts)
        
    time_entries = [x for x in time_entries if 'pid' in x]
        
    for time_entry in time_entries:
        project_id = time_entry['pid']
        if project_id not in project_id_names:
            project_id_names[project_id] = fetch_project(project_id)
        time_entry['project_name'] = project_id_names[project_id]['name']
        
        time_entry['start'] = datetime.fromisoformat(time_entry['start'])
        if 'stop' in time_entry:
            time_entry['stop'] = datetime.fromisoformat(time_entry['stop'])
        if time_entry['duration'] < 0:
            time_entry = update_current_time_entry_duration(time_entry)        
    return time_entries

def get_day_entries(date):
    # TODO
    start_date = datetime.fromisoformat(f'{str(date)}T01:00:00+00:00')
    end_date = datetime.fromisoformat(f'{str(date)}T23:59:59+00:00')
    return get_time_entries(start_date, end_date)

def get_todays_entries():
    return get_day_entries(date.today())

def get_week_entries():
    pass

def get_month_entries():
    pass

def get_last_n_days(n):
    entries = []
    for i in range(n):
        entries.append(get_day_entries(date.today() - dt.timedelta(n-1-i)))
    return entries

def get_all_data():
    pass 

## Time entry processing

In [8]:
def accumulate(entries):
    times = {}
    for entry in entries:
        times[entry['project_name']] = times.get(entry['project_name'], 0) + entry['duration']
    return sorted(list(times.items()), key=itemgetter(1))

def n_acc(n, sleep=False):
    entries = get_last_n_days(n)
    entries = list(map(accumulate, entries))
    return entries

def filter_sleep(entries):
    "The expected format is [[]]. Each list element at depth 1 represents a day. A list item at depth two represents a time entry."
    return [[(k,v) for k,v in x if k != 'sleep睡眠'] for x in entries]

# Goals

In [9]:
print("Projects:")
for k in project_id_names:
    print(k, project_id_names[k]['name'])
print()

Projects:
185499583 morning
163125874 improvements
165777572 social productive社会的、生産的
186239363 evening
163125866 sleep睡眠
187567761 unknown
165947128 entertainment
165352471 communicationコミュニケーション
165598846 cook食べ物
165297879 maintenanceメンテナンス
187511108 nonfiction, listen
163125865 meditate瞑想
184591500 commute
185247273 write
165296570 toiletトイレ
165304788 sportトレニング
163126231 eat食べる
165296445 social社会的
186567117 life_debugging
165297234 plan計画
163125846 tulpa_training
165301785 AIA_original
187084115 aia_discussion
186119188 reflect
165296497 research研究
185725569 frisky
186454695 SERI_MATS_event
185927511 AIA_skillup
165354205 shopping買い物
187650764 N87
169322498 music_organization
178004458 youtube-produce
169680640 reading_non-fiction
165302994 nap睡眠_nap
165335481 showerシャワー
186291374 AIA_communicate
187708034 improvement, plan
165452212 idea_formulation
165613431 music_discovery
182726911 find_flat
168113610 learn_steno
187852475 read, alignment forum
187853378 write, alignment forum


In [10]:
print("Daily goals, given as project_id and minutes:")
daily_goals = [
               (186119188,  5), # Reflect           
               (163125865, 40), # Meditate           
               (165304788, 30), # Sport
               (163125846,  5), # Tulpa training
               (165301785, 90), # AIA original                   
               (185927511, 20), # AIA_skillup
               (186291374, 30), # AIA_explain
               (187852475, 30), # read
               (168113610, 10), # Steno
             ]
daily_goals = [(pid, project_id_names[pid]['name'], mins*60) for pid, mins in daily_goals]
for i in daily_goals:
    print(i)
print()
daily_goals.reverse()

Daily goals, given as project_id and minutes:
(186119188, 'reflect', 300)
(163125865, 'meditate瞑想', 2400)
(165304788, 'sportトレニング', 1800)
(163125846, 'tulpa_training', 300)
(165301785, 'AIA_original', 5400)
(185927511, 'AIA_skillup', 1200)
(186291374, 'AIA_communicate', 1800)
(187852475, 'read, alignment forum', 1800)
(168113610, 'learn_steno', 600)



# Visualisations

In [11]:
def general_axis(ax):
    ax.set_facecolor('white')
    return ax

In [12]:
def cumulative_plot(axs, data, daily_goals, limit_y=True):
    text_size = 14
    plot_y_lim = 1.2
    axs.grid(which='both')
    general_axis(axs)
    day = data[0]
    day_d = dict(day)
    percents = [np.array([day_d.get(name, 0) / seconds / len(data) for _,name,seconds in daily_goals])]
    for i, day in enumerate(data[1:]):
        day_d = dict(day)
        percents.append(np.array(percents[-1]) + np.array([day_d.get(name, 0) / seconds / len(data) for _,name,seconds in daily_goals]))
    percents = np.array(percents)
    for i in range(percents.shape[1]):
        axs.plot(np.arange(-percents.shape[0]+1, 1), percents[:,i], label=daily_goals[i][1], linewidth=7.5, alpha=0.5)
        target = percents[-2,i] + (1 / (len(percents) - 1))
        color = 'red' if percents[-1,i] < target else 'lime'
        axs.plot([-1, 0], [percents[-2,i], target], linestyle='dashed', color=color, alpha=0.5)
        if not limit_y:
            axs.text(0, percents[-1,i], str(daily_goals[i][1]), va='center')
        else:
            for j in range(percents.shape[0]):
                if percents[j,i] > plot_y_lim and j > 0 and percents[j-1,i] < plot_y_lim:
                    axs.text(-7+j, percents[j-1,i], str(daily_goals[i][1]), va='center')
                    break
            else:
                if percents[j-1,i] < plot_y_lim:
                    axs.text(0, percents[-1,i], str(daily_goals[i][1]), va='center')
                
    axs.plot(0,1,marker=4)
    axs.plot([-6,0],[0,1], linestyle='dashed', label='optimal', linewidth=3, alpha=0.4)
    if limit_y:
        axs.set_ylim(-0.03, plot_y_lim)
    axs.legend()

def generate_cumulative_plot(show_plot=True):
    text_size = 14
    scale = 1.5*1.41
    
    fig, axs = plt.subplots(1,1,figsize=(16*scale,9*scale))
    figtitle = "0_cumulative_goals_1"
    fig.suptitle(figtitle, fontsize=text_size*3)
    cumulative_plot(axs, acc_7, daily_goals)
    plt.savefig(f"plots/{figtitle}.svg", facecolor='white', transparent=False, bbox_inches='tight')
    
    fig, axs = plt.subplots(1,1,figsize=(16*scale,9*scale))
    figtitle = "0_cumulative_goals_2_no_ylimit"
    fig.suptitle(figtitle, fontsize=text_size*3)
    cumulative_plot(axs, acc_7, daily_goals, limit_y=False)
    plt.savefig(f"plots/{figtitle}.svg", facecolor='white', transparent=False, bbox_inches='tight')
    
    if show_plot:
        plt.show()

In [13]:
def goal_plot(axs, daily_goals, acc):
    text_size = 14
    red_percent = 5
    y_pos = np.arange(len(daily_goals))
    d_acc = dict(acc)
    percents = [max(d_acc.get(name, 0) / seconds * 100, 1) for _,name,seconds in daily_goals]
    general_axis(axs)
    axs.grid(axis='x')
    color = []
    for p in percents:
        color.append('red' if p < red_percent else ('orange' if p < 100 else 'lime'))
    axs.barh(y_pos, percents, color=color)
    axs.tick_params(axis='x', labelsize=text_size)
    axs.set_yticks(y_pos, labels=[name for _,name,_ in daily_goals], size=text_size)
    axs.axvline(red_percent, color='orange')
    axs.axvline(100, color='green')
    axs.set_xlim(0, 115)
    for y, (_,name,target) in zip(y_pos, daily_goals):
        axs.text(3, y, f"{dt.timedelta(seconds=d_acc.get(name, 0))} / {dt.timedelta(seconds=target)}", size=text_size)
    return axs

def generate_goal_plot(show_plot=True):
    scale = 1.5
    text_size = 14
    fig, axs = plt.subplots(3, 2, figsize=(17*scale, 9*scale))
    figtitle = "1_goals"
    fig.suptitle(figtitle, fontsize=text_size*3)
    axs = axs.flatten()
    for i, a in enumerate(reversed(acc_7[1:])):
        goal_plot(axs[i], daily_goals, a)
        date = datetime.now() - dt.timedelta(i)
        axs[i].set_title(date.strftime('%Y-%m-%d %A'))
    plt.savefig(f"plots/{figtitle}.svg", facecolor='white', transparent=False, bbox_inches='tight')
    if show_plot:
        plt.show()

In [14]:
def bar(ax, data):
    ax.grid(axis='y')
    ax.set_axisbelow(True)
    for j, d in enumerate(data): 
        height = 0
        cmap = plt.cm.get_cmap('hsv', len(d))
        inverse_shrink = 5
        for i, (x,y) in enumerate(reversed(d)):
            y_scaled = y/(60*60)
            new_height = height + y_scaled
            ax.bar(j, height=y_scaled, tick_label='Today', alpha=1, color=cmap(i), bottom=height, edgecolor='black')
            ax.text(j, new_height, str(dt.timedelta(seconds=y)) + ' ' + x, va='top', ha='center', size=8)
            height = new_height
    return ax

def generate_bars(show_plot=True):
    text_size = 14
    scale = 1.5*1.41
    fig, axs = plt.subplots(1,1,figsize=(16*scale,9*scale))
    figtitle = "2_bar_1"
    fig.suptitle(figtitle, fontsize=text_size*3)
    bar(axs, acc_7)
    axs.axhline(24, color='red')
    plt.savefig(f"plots/{figtitle}.svg", facecolor='white', transparent=False, bbox_inches='tight')
    if show_plot:
        plt.show()
    fig.clf()
    del fig
    gc.collect()

    fig, axs = plt.subplots(1,1,figsize=(16*scale,9*scale))
    figtitle = "2_bar_2_no-sleep"
    fig.suptitle(figtitle, fontsize=text_size*3)
    bar(axs, acc_7_nosleep)
    plt.savefig(f"plots/{figtitle}.svg", facecolor='white', transparent=False, bbox_inches='tight')
    if show_plot:
        plt.show()

In [15]:
def pie(ax, data):
    names, sizes = [x[0] for x in data], [x[1] for x in data]
    names = [f"{name} {math.floor(size / (60*60))}:{math.floor(size / 60 % 60):02}" for name, size in zip(names, sizes)]
    ax.pie(sizes, labels=names, autopct='%1.1f%%',
            shadow=True, startangle=180-45, explode=0.05 * np.random.rand(len(data)))
    return ax

def generate_pies(show_plot=True):
    text_size = 14
    fig, axs = plt.subplots(1,1,figsize=(20,20))
    figtitle = "3_pie_1"
    fig.suptitle(figtitle, fontsize=text_size*3)
    pie(axs, acc_7[-1])
    plt.savefig(f"plots/{figtitle}.svg", facecolor='white', transparent=False, bbox_inches='tight')
    if show_plot:
        plt.show()
    fig.clf()
    del fig
    gc.collect()

    fig, axs = plt.subplots(1,1,figsize=(20,20))
    figtitle = "3_pie_2_no-sleep"
    fig.suptitle(figtitle, fontsize=text_size*3)
    pie(axs, acc_7_nosleep[-1])
    plt.savefig(f"plots/{figtitle}.svg", facecolor='white', transparent=False, bbox_inches='tight')
    if show_plot:
        plt.show()
    fig.clf()
    del fig
    gc.collect()

In [16]:
# Continuously fetch the data and update the plots
WAIT = 90
SHOW_PLOTS = False

while True:
    try:
        acc_7 = n_acc(7)
        acc_7_nosleep = filter_sleep(acc_7)
        print('data fetch complete')
        break
    except ConnectionError:
        print(f"Connection error. Sleeping {WAIT}s")
        time.sleep(WAIT)
clear_output()
generate_goal_plot(show_plot=SHOW_PLOTS)
generate_pies(show_plot=SHOW_PLOTS)
generate_bars(show_plot=SHOW_PLOTS)
generate_cumulative_plot(show_plot=SHOW_PLOTS)
plt.close('all')
print(f"\r last generation: {datetime.now()}",end="")

 last generation: 2022-12-10 16:51:36.280476